# 📰 AI-Based Online News Bias Detection System (Gemini-Only, Multi-Agent, with Sample Articles)

This notebook implements an **AI-based Online News Bias Detection System** using a **multi-agent architecture** that relies **only on Gemini (via `google-genai`)**.

Enhancements in this version:

- ✅ Pre-written **sample news articles** for quick testing  
- ✅ Optional **dropdown-style selection** (where `ipywidgets` is available)  
- ✅ Existing **manual** and **file-based** input modes

No local dataset or traditional ML model is required.


## 1. Environment Setup

Install or update the **Google Gen AI SDK** (`google-genai`) used to call Gemini models.


In [1]:
# 🔧 Install / upgrade Google Gen AI SDK

# Remove deprecated google-generativeai if present
!pip uninstall -y google-generativeai -q || print("google-generativeai not installed")

# Install the new official SDK
!pip install -q google-genai

/bin/bash: -c: line 1: syntax error near unexpected token `"google-generativeai not installed"'
/bin/bash: -c: line 1: `pip uninstall -y google-generativeai -q || print("google-generativeai not installed")'


## 2. Imports & Gemini Configuration

We import the required libraries and configure the Gemini client.


In [2]:
import os
import textwrap
from getpass import getpass
from pathlib import Path

from google import genai
from google.genai import types

# Optional: widgets for dropdown selection (if available)
try:
    import ipywidgets as widgets
    from IPython.display import display
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False

### 2.1 Configure Gemini API

You will be prompted once per session for your **Gemini API key**.


In [3]:
# 🔑 Get Gemini API key (once per session)
if "GEMINI_API_KEY" not in os.environ:
    os.environ["GEMINI_API_KEY"] = getpass("Paste your GEMINI API key (input hidden): ")

# 🤝 Create Gemini client
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# 🤖 Choose a Gemini model
GEMINI_MODEL_NAME = "gemini-2.0-flash-001"

print("Gemini client initialised with model:", GEMINI_MODEL_NAME)

Paste your GEMINI API key (input hidden): ··········
Gemini client initialised with model: gemini-2.0-flash-001


### 2.2 Helper – Gemini Call Wrapper

A small helper to call Gemini and return text output.


In [4]:
def call_gemini(prompt: str,
                model_name: str = GEMINI_MODEL_NAME,
                temperature: float = 0.2,
                max_output_tokens: int = 2048) -> str:
    """Call the Gemini model with a text prompt and return the response text."""
    try:
        response = client.models.generate_content(
            model=model_name,
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=temperature,
                max_output_tokens=max_output_tokens,
            ),
        )
        return response.text or ""
    except Exception as e:
        return f"[Gemini call failed: {e}]"

## 3. Sample News Articles Library

To make testing easier, we provide a small library of **sample articles** that simulate different types of bias.

You can use these samples instead of pasting your own text every time.


In [5]:
SAMPLE_ARTICLES = [
    {
        "id": 0,
        "name": "Economic Reform – Mixed Perspectives",
        "title": "Government unveils controversial economic reform package",
        "content": (
            "The government today announced a sweeping economic reform package that it claims will "
            "boost long-term growth and competitiveness. Supporters within the ruling party praised "
            "the plan as a bold step towards modernising the economy. However, opposition leaders "
            "slammed the reforms as a giveaway to large corporations and wealthy donors, arguing that "
            "ordinary workers will bear the brunt of the changes. Critics also expressed concerns "
            "about reduced funding for public services, calling the measures short-sighted and ideologically driven."
        ),
    },
    {
        "id": 1,
        "name": "Climate Policy – Left-Leaning Tone",
        "title": "Experts warn government's weak climate plan puts future generations at risk",
        "content": (
            "A coalition of climate scientists and campaigners has condemned the government's latest climate plan "
            "as 'dangerously inadequate'. They argue that the continued subsidies for fossil fuel companies and the "
            "lack of binding emissions targets reveal a shocking disregard for the environment. Activists insist that "
            "only a rapid transition to renewable energy, coupled with strict regulations on polluters, can prevent "
            "catastrophic climate breakdown. They accuse ministers of bowing to corporate interests instead of listening "
            "to the overwhelming scientific evidence."
        ),
    },
    {
        "id": 2,
        "name": "Security & Immigration – Right-Leaning Tone",
        "title": "Government finally gets tough on illegal immigration, say supporters",
        "content": (
            "The government has introduced a new set of tough measures aimed at tackling illegal immigration, "
            "winning praise from supporters who say the country's borders have been too lax for too long. The plan "
            "includes faster deportations and stricter checks at entry points. Ministers argue that these steps are "
            "necessary to restore order and protect hard-working citizens. Critics, however, claim the policy is harsh "
            "and risks undermining the rights of vulnerable people seeking safety."
        ),
    },
    {
        "id": 3,
        "name": "Neutral Business Report – Low Bias",
        "title": "Local businesses report steady growth in quarterly earnings",
        "content": (
            "A survey of local businesses indicates steady growth in quarterly earnings across a range of sectors, "
            "including retail, manufacturing, and technology. Analysts attribute the positive results to gradual increases "
            "in consumer spending and improved supply chain stability. Several business owners noted that while challenges "
            "remain, particularly around energy costs and staffing, the overall outlook for the next quarter is cautiously optimistic."
        ),
    },
]


def list_sample_articles() -> None:
    """Print available sample articles with their IDs and names."""
    print("Available sample articles:")
    for art in SAMPLE_ARTICLES:
        print(f"  [{art['id']}] {art['name']}")


def get_sample_article_by_id(article_id: int) -> dict:
    """Retrieve a sample article by its ID."""
    matches = [a for a in SAMPLE_ARTICLES if a["id"] == article_id]
    if not matches:
        raise ValueError(f"No sample article with id={article_id}")
    return matches[0]

## 4. Multi-Agent Architecture (Gemini-Only)

We define the following agents:

1. `NewsInputAgent` – handles article input (manual, file, or sample selection).  
2. `PreprocessingAgent` – cleans and normalises the text.  
3. `GeminiBiasAnalysisAgent` – sends the article to Gemini for bias analysis.  
4. `ExplanationAndReportingAgent` – formats and displays the final report.  
5. `BiasDetectionCoordinator` – orchestrates the full workflow.


### 4.1 Agent 1 – `NewsInputAgent`

**Role:** Ingest a news article from one of three modes:

- Manual input (title + content)  
- `.txt` file upload/path  
- Predefined **sample article** (via ID or dropdown)


In [6]:
class NewsInputAgent:
    """Handles ingestion of news article text from various input sources."""

    def from_manual_input(self) -> dict:
        """Ask the user to paste or type the article title and body."""
        print("✏️ Manual input mode selected.")
        title = input("Enter article title (optional, press Enter to skip): ").strip()
        print("\nPaste or type the article content below. End with an empty line:")
        lines = []
        while True:
            line = input()
            if line.strip() == "":
                break
            lines.append(line)
        content = "\n".join(lines).strip()
        return {
            "title": title if title else "[Untitled article]",
            "content": content,
        }

    def from_text_file(self) -> dict:
        """Upload or read a .txt file and return its content as an article."""
        try:
            # Colab-style upload
            from google.colab import files  # type: ignore
            print("📂 Please upload a .txt file containing the news article text:")
            uploaded = files.upload()
            if not uploaded:
                raise ValueError("No file uploaded.")
            filename = next(iter(uploaded.keys()))
            print(f"✅ Uploaded: {filename}")
            path = Path(filename)
        except ImportError:
            # Local Jupyter path-based input
            path_str = input("Enter path to a .txt file: ").strip()
            path = Path(path_str)
            if not path.exists():
                raise FileNotFoundError(f"File not found: {path}")

        with open(path, "r", encoding="utf-8") as f:
            content = f.read()
        title = path.stem.replace("_", " ").title()
        return {
            "title": title,
            "content": content.strip(),
        }

    def from_sample_article(self, article_id: int | None = None) -> dict:
        """Select a sample article by ID (or interactively if not provided)."""
        if article_id is None:
            # If widgets are available, use a dropdown selection
            if WIDGETS_AVAILABLE:
                print("🧪 Sample article selection (dropdown):")
                options = [(a["name"], a["id"]) for a in SAMPLE_ARTICLES]
                dropdown = widgets.Dropdown(
                    options=options,
                    description="Sample:",
                    value=SAMPLE_ARTICLES[0]["id"],
                    disabled=False,
                )
                display(dropdown)
                print("Adjust the dropdown, then run this cell again with the chosen value, or pass article_id explicitly.")
                # Return the default for now
                chosen = get_sample_article_by_id(dropdown.value)
            else:
                print("🧪 Widgets not available. Please choose a sample by ID:")
                list_sample_articles()
                while True:
                    try:
                        raw = input("Enter sample ID: ").strip()
                        article_id = int(raw)
                        chosen = get_sample_article_by_id(article_id)
                        break
                    except Exception as e:
                        print(f"Invalid ID or error: {e}. Try again.")
        else:
            chosen = get_sample_article_by_id(article_id)

        return {
            "title": chosen["title"],
            "content": chosen["content"],
        }

### 4.2 Agent 2 – `PreprocessingAgent`

**Role:** Clean and normalise the article text before sending it to Gemini.

It can:

- Strip whitespace and control characters.  
- Optionally truncate extremely long text to a safe length.  
- Combine title + body in a consistent format.


In [7]:
class PreprocessingAgent:
    """Cleans and prepares article text for Gemini analysis."""

    def __init__(self, max_chars: int = 8000):
        self.max_chars = max_chars

    def preprocess(self, article: dict) -> dict:
        title = article.get("title", "").strip()
        content = article.get("content", "").strip()

        # Normalise whitespace
        content = "\n".join(line.strip() for line in content.splitlines())
        content = content.strip()

        # Truncate if too long
        if len(content) > self.max_chars:
            content = content[: self.max_chars] + "\n...[TRUNCATED FOR ANALYSIS]"

        return {
            "title": title if title else "[Untitled article]",
            "content": content,
        }

### 4.3 Agent 3 – `GeminiBiasAnalysisAgent`

**Role:** Call Gemini with a carefully crafted prompt to:

- Classify political orientation (left/centre/right/other).  
- Rate bias intensity (low/medium/high).  
- Highlight emotionally charged language and framing.  
- Provide a neutral summary.


In [8]:
class GeminiBiasAnalysisAgent:
    """Uses Gemini to analyse news article bias and framing."""

    def analyse(self, article: dict) -> str:
        title = article.get("title", "").strip()
        content = article.get("content", "").strip()

        prompt = f"""You are an expert in media bias, political communication, and journalism ethics.

You will analyse the political bias and framing of the following news article.

ARTICLE TITLE:
{title}

ARTICLE BODY:
{content}

Your tasks:
1. Classify the political orientation of the article as one of:
   - left
   - centre
   - right
   - other / unclear

2. Rate the overall bias intensity as one of:
   - low
   - medium
   - high

3. Explain your reasoning in 3–6 bullet points. Refer explicitly to phrases, framing choices, or omissions that contribute to bias.

4. Identify any emotionally charged or loaded language, and briefly explain why it could be considered biased or persuasive.

5. Provide a short, neutral summary of the article in 2–3 sentences, written in as unbiased a tone as possible.

Format your response clearly with headings such as:
- "Bias Classification"
- "Bias Intensity"
- "Evidence and Reasoning"
- "Loaded Language"
- "Neutral Summary"
"""
        return call_gemini(prompt)

### 4.4 Agent 4 – `ExplanationAndReportingAgent`

**Role:** Format the collected information into a structured, readable bias report.


In [9]:
class ExplanationAndReportingAgent:
    """Formats and prints a structured bias report based on Gemini's analysis."""

    def generate_report(self, article: dict, gemini_output: str) -> None:
        title = article.get("title", "").strip()
        content = article.get("content", "").strip()

        print("📰 ARTICLE TITLE:")
        print(title if title else "[Untitled article]")
        print("\n" + "-" * 80)
        print("📄 ARTICLE PREVIEW (first ~500 characters):")
        preview = content[:500].replace("\n", " ")
        print(preview + ("..." if len(content) > 500 else ""))
        print("\n" + "-" * 80)
        print("🧠 GEMINI BIAS ANALYSIS REPORT:")
        print(textwrap.dedent(gemini_output))
        print("\n" + "-" * 80)
        print("End of report.")

### 4.5 Coordinator – `BiasDetectionCoordinator`

**Role:** Orchestrate the full Gemini-only workflow.

Supports three modes:

- `"manual"` – paste or type article text  
- `"file"` – read from `.txt` file  
- `"sample"` – pick one of the predefined sample articles


In [10]:
class BiasDetectionCoordinator:
    """Coordinates the Gemini-only bias detection workflow."""

    def __init__(self):
        self.input_agent = NewsInputAgent()
        self.preproc_agent = PreprocessingAgent()
        self.gemini_agent = GeminiBiasAnalysisAgent()
        self.report_agent = ExplanationAndReportingAgent()

    def run(self, mode: str = "manual", sample_id: int | None = None) -> None:
        """Run the full workflow.

        mode: "manual", "file", or "sample"
        sample_id: optional ID for sample mode (if None, interactive selection is used)
        """
        if mode not in {"manual", "file", "sample"}:
            raise ValueError("Mode must be 'manual', 'file', or 'sample'.")

        # 1. Ingest
        if mode == "manual":
            article = self.input_agent.from_manual_input()
        elif mode == "file":
            article = self.input_agent.from_text_file()
        else:  # sample
            article = self.input_agent.from_sample_article(article_id=sample_id)

        # 2. Preprocess
        processed_article = self.preproc_agent.preprocess(article)

        # 3. Gemini analysis
        print("🧠 Calling Gemini for bias analysis...\n")
        gemini_output = self.gemini_agent.analyse(processed_article)

        # 4. Reporting
        self.report_agent.generate_report(processed_article, gemini_output)

## 5. Running the Gemini-Only News Bias Detection

You can now instantiate the coordinator and choose one of the modes:

- `"sample"` – use one of the built-in sample articles (recommended for quick testing).  
- `"manual"` – type/paste your own article.  
- `"file"` – upload or reference a `.txt` file with article content.


In [11]:
# Instantiate coordinator
coordinator = BiasDetectionCoordinator()

### 5.1 Option A – Use a Sample Article (Recommended for Quick Testing)

First, list available samples, then run with `mode="sample"`.


In [12]:
# List the built-in sample articles
list_sample_articles()

# Run full workflow using a sample article by ID (e.g. 0, 1, 2, or 3)
# Change sample_id to try different examples.
coordinator.run(mode="sample", sample_id=0)

Available sample articles:
  [0] Economic Reform – Mixed Perspectives
  [1] Climate Policy – Left-Leaning Tone
  [2] Security & Immigration – Right-Leaning Tone
  [3] Neutral Business Report – Low Bias
🧠 Calling Gemini for bias analysis...

📰 ARTICLE TITLE:
Government unveils controversial economic reform package

--------------------------------------------------------------------------------
📄 ARTICLE PREVIEW (first ~500 characters):
The government today announced a sweeping economic reform package that it claims will boost long-term growth and competitiveness. Supporters within the ruling party praised the plan as a bold step towards modernising the economy. However, opposition leaders slammed the reforms as a giveaway to large corporations and wealthy donors, arguing that ordinary workers will bear the brunt of the changes. Critics also expressed concerns about reduced funding for public services, calling the measures shor...

-------------------------------------------------------

### 5.2 Option B – Manual Input Mode

Run the cell below and follow the prompts to paste or type an article.


In [16]:
# Run full workflow in manual mode
coordinator.run(mode="manual")

✏️ Manual input mode selected.
Enter article title (optional, press Enter to skip): South Africa dismissed for lowest-ever T20 total of 74 as India win series opener by 101 runs in Cuttack

Paste or type the article content below. End with an empty line:
In the first of five T20Is between India and South Africa, hosts secure victory in Cuttack with Hardik Pandya to fore, hitting unbeaten 59 off 28 balls; South Africa bowled out for just 74 - their lowest in T20 cricket; India won ODI series vs South Africa 2-1 after losing Test series 2-0
In the first of five T20Is between India and South Africa, hosts secure victory in Cuttack with Hardik Pandya to fore, hitting unbeaten 59 off 28 balls; South Africa bowled out for just 74 - their lowest in T20 cricket; India won ODI series vs South Africa 2-1 after losing Test series 2-0

🧠 Calling Gemini for bias analysis...

📰 ARTICLE TITLE:
South Africa dismissed for lowest-ever T20 total of 74 as India win series opener by 101 runs in Cuttack

--

### 5.3 Option C – File Input Mode

Prepare a `.txt` file with the full article text, then run the cell below:

- In **Colab**: you will be prompted to upload the file.  
- In **local Jupyter**: you will be asked for the file path.


In [14]:
# Run full workflow in file mode
# coordinator.run(mode="file")